# Laboratório 2 - fast.ai DL1 2019 Brasília

Bem-vindos ao laboratório 2 da turma fast.ai DL1 2019 Brasília. O objetivo do laboratório é consolidar e expandir conhecimentos obtidos nas aulas 3 e 4. Mas principalmente o conteúdo de NLP(Natural Language Processing) ou PLN(processamento de linguagem natural).

O principal desse laboratório é o treinamento de um classificador de textos de reclamações para companhias aéreas. Os textos serão classificados de acordo com o seu tema. Esses são dados reais da ANAC e portanto não temos um benchmark oficial para verificar se os resultados são bons ou ruins. Se atenha a esse notebook como um guia de como utilizar a fastai, e por consequência o ULMFIT, para classificação de textos.

## Importação de bibliotecas

In [10]:
from fastai.text import *
import pandas as pd
from pathlib import Path
from sklearn.metrics import classification_report

In [11]:
#ajustando os caminhos que serão utilizados

datapath = Path('../data')
modelspath = Path('../models')
path = Path('../')
best_model_path = Path('../models/bestmodel30k')

## Downloads

Download dos dados de classificação com as reclamações

In [5]:
#Criar as pastas
!mkdir ../data/
!mkdir ../models
# Download dos dados
!curl https://storage.googleapis.com/gde-dl-bsb/data/reclamacoes.csv -o ../data/reclamacoes.csv
# Download do modelo em portugues pretreinado na wikipedia
!curl https://storage.googleapis.com/gde-dl-bsb/models/bestmodel30k.pth -o ../models/bestmodel30k.pth
# Download do vocabulário
!curl https://storage.googleapis.com/gde-dl-bsb/models/itos.pkl -o ../models/itos.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  516k  100  516k    0     0  1276k      0 --:--:-- --:--:-- --:--:-- 1278k


## Importar dados

In [12]:
df = pd.read_csv(datapath.joinpath('reclamacoes.csv'), sep=';')

In [13]:
df.head()

,manifestacao_clean,TEMA,SUBTEMA
0,Olá em 26/06 comprei o localizador 8RK87X para...,Alteração pela empresa aérea,"Alternativas (reacomodação, reembolso integral..."
1,"Prezados, Abri o processo junto a LATAM pra te...",Transporte de bagagem,Extravio
2,"No dia 17/12 comprei uma passagem, Vitória x C...",Alteração pelo passageiro,Remarcação e resilição
3,Estou tendo um problema! Recentemente fiz a co...,Programas de Fidelidade,Programas de Fidelidade
4,"Olá, Minha mala retornou da viagem de Recife c...",Transporte de bagagem,Avaria


### Exemplos

In [14]:
df.sample().values[0]

array(['Fiz a reserva do Hotel Barceló Bávaro Palace (Solicitação de compra 13480028100) a partir do dia 16, até o dia 23 de outubro (Anexo 1). Comprei a passagem para Punta Cana para o dia 16 com a chegaga em Punta Cana as 17:37 desse dia (Anexo 2). Houve uma alteração no voo pela Avianca. Fomos informados que esta alteração deveu-se a uma greve de Pilotos da Avianca, que estava ocorrendo em Bogotá. Partimos do Rio de Janeiro na segunda-feira (Anexo 3) e pernoitamos em Bogotá no Hotel Capital (Anexo 4). Partimos de Bogotá para Punta Cana no dia 17 (Anexo 5), lá chegando as 12:21 h (Anexo 4). Portanto pagamos a diária da segunda-feira dia 16 de outubro e só ingressamos no hotel na terça-feira dia 17 de outubro. Tendo em vista que esta viagem foi programada para gozo de férias com a família [Eu, minha esposa, filho e Nora, e o transtorno gerado com o atraso do voo e a consequente perda de um dia de nossas férias, solicito que seja feita a devolução do valor da diária paga do dia 16 para

In [15]:
df.groupby('TEMA').count()

,manifestacao_clean,SUBTEMA
TEMA,,
Alteração pela empresa aérea,4340,4340
Alteração pelo passageiro,3992,3992
Assistência ao PNAE,386,386
Check-in e embarque,2311,2311
Execução do voo,5097,5097
Oferta e compra,4914,4914
Outros,2483,2483
Programas de Fidelidade,3521,3521
Reclamações contra valores e regras do contrato,5165,5165


## Pré-requisitos


Objetos necessários antes de treinar o modelo. Tokenizador para realizar a separação das palavras e a importação do vocabulário presente na rede pré-treinada

In [15]:
#tokenizador
tokenizer = Tokenizer(lang='pt', n_cpus=8)
#vocabulario
with modelspath.joinpath('itos.pkl').open('rb') as f:
    itos = pickle.load(f)
vocab = Vocab(itos)

### Estrutura do ULMFIT

![ULMFIT](../images/Figure_5.png)

## Embeddings

Word Embeddings são representações multi-dimensionais das palavras e estes são treinados a partir de uma tarefa simples. No nosso caso prever a próxima palavra com o Language Model

[projector](https://projector.tensorflow.org/)

## Language Model

### Criar o TextLMDataBunch para realizar o transfer learning

Gerar o TextLMDataBunch. Isso significa importar os dados e realizar o preprocessamento dos deles para se adequar ao que a rede neural espera

In [ ]:
train_bool = np.random.rand(len(df)) < 0.8

In [ ]:
data_lm = TextLMDataBunch.from_df(path,
                                  train_df= df[train_bool],
                                  valid_df= df[~train_bool],
                                  tokenizer=tokenizer,
                                  vocab=vocab,
                                  text_cols=0,
                                  bs=24,
                                  max_vocab=35000)

In [ ]:
data_lm.save('data_lm')

### Transfer Learning

In [ ]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, pretrained_fnames=(best_model_path,modelspath.joinpath('itos')))

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, max_lr=5e-2)

In [ ]:
learn.save_encoder('pretrained_encoder')

## Classificador

### Criar o TextClasDataBunch para os dados de classificação

In [ ]:
data_clas = TextClasDataBunch.from_df(path,
                                      train_df= df[train_bool],
                                      valid_df= df[~train_bool],
                                      tokenizer=tokenizer,
                                      text_cols=0,
                                      bs=24,
                                      vocab=vocab,
                                      max_vocab=35000,
                                      label_cols=1)

In [ ]:
data_clas.show_batch()

In [ ]:
clf = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False)

In [ ]:
clf.fit_one_cycle(5, 1e-2)

### Verificar a acurácia

In [ ]:
val_probs, val_targets = clf.get_preds()

In [ ]:
val_preds = torch.argmax(val_probs,1)

In [ ]:
print(classification_report(val_targets, val_preds, target_names=data_clas.valid_dl.y.classes))

### Classificação de outros exemplos

In [ ]:
clf.predict('Minha bagagem foi desviada quero que recuperem imediatamente')

### Colocar o modelo em produção 

In [ ]:
clf.export()